In [1]:
import lda_clust

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
x = lda_clust.simulate_data(D=10, K=3, gamma=5, command_level_topics=False, secondary_topic=False, V=10, csi=10, omega=5)

In [3]:
m = lda_clust.topic_model(W=x['w'], K=3, fixed_K = True, V=10, fixed_V=True, secondary_topic = True, 
            command_level_topics=True, H=3, fixed_H=True, gamma=1.0, eta=1.0)

In [4]:
m.gensim_init()

In [5]:
m.z

{0: {0: array([0, 0, 0, 1, 1, 0, 0, 1]),
  1: array([0, 0, 0, 0, 1, 1, 0, 1, 0]),
  2: array([1, 0, 1, 0, 0, 0, 0, 0]),
  3: array([1, 1]),
  4: array([1, 0]),
  5: array([1, 0, 0, 0]),
  6: array([0, 0, 1, 1, 0, 0, 1]),
  7: array([0, 0, 0, 0, 0, 0, 1, 0]),
  8: array([0, 0, 1, 0, 0]),
  9: array([1, 0, 1, 0, 1, 1]),
  10: array([0, 0, 0, 1, 0, 0]),
  11: array([0, 0, 0, 0, 1, 0])},
 1: {0: array([0, 1, 1, 1, 0, 1]),
  1: array([1, 1, 1, 0, 1, 0, 0, 0]),
  2: array([1, 1, 0, 1, 0, 1, 1, 0]),
  3: array([1, 0, 0, 1, 1]),
  4: array([1, 1, 0, 1, 1, 1]),
  5: array([0, 0, 1, 1, 1, 0]),
  6: array([1, 1, 0, 1]),
  7: array([0, 1, 1, 1]),
  8: array([0, 1, 0, 0, 0, 1]),
  9: array([0, 0, 0, 1, 1, 1, 0])},
 2: {0: array([0, 1, 0, 1, 1, 0]),
  1: array([0, 1, 1, 0, 0, 0]),
  2: array([0, 0, 1, 1, 1, 0, 1, 1, 0, 1]),
  3: array([1, 1, 0, 0, 1, 1]),
  4: array([1, 1, 0]),
  5: array([1, 1, 1, 0]),
  6: array([0, 1, 1]),
  7: array([1, 0, 1, 0, 1, 0, 1, 1, 1]),
  8: array([1, 1, 1])},
 3: {0: a